#### Capstone 2 Arima Model

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib  as mpl 
%matplotlib inline

# Load Specific forecasting tools
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults, ARIMA, ARIMAResults
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima

import itertools
import statsmodels.api as sm
plt.style.use('fivethirtyeight')
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# Ignore harmless warnings
import warnings
warnings.filterwarnings('ignore')

#### DATA CLEANING AND EXPLORATORY DATA ANALYSIS

In [ ]:
#get data
eurusd = pd.read_csv(r'eurousd.csv', index_col=0, parse_dates=True, skipinitialspace=True)

In [ ]:
# Examine firt 5 rows
eurusd.head()

In [ ]:
# Examine last five rows
eurusd.tail()

In [ ]:
# Examine index
eurusd.index

In [ ]:
# Examine data types
eurusd.dtypes

In [ ]:
# Select column 'Open' for analysis
eurusdopen = eurusd.Open

In [ ]:
# Get summary statistics
eurusdopen.describe() 

In [ ]:
# Check for missing data
print(eurusd.isnull().sum())

There are no missing values in the data set

In [ ]:
# Plot the exchange rate for 'open column'

eurusd['Open'].plot(ylim=[0,1.75], linewidth=2, figsize = (12,5)).autoscale(axis='x', tight=True)
plt.title('Exchange Rate Chart', size=20)
plt.xlabel('Years', size=17)
plt.ylabel('Exchange Rate', size=17)
plt.autoscale(enable=True, axis='x', tight=True)
plt.tick_params(labelsize=14)

In [ ]:
# Carry out Decomposition

seasonal = seasonal_decompose(eurusd.Open, model='additive', freq=52)
seasonal.plot()
mpl.rcParams['figure.figsize'] = 12,6

STATIONARITY

In [ ]:
# Examine the trend of the mean and standard deviation to examine impact on exchange rate trend

eurusd['50 daily-EMA(Open)'] = eurusd['Open'].rolling(window=50).mean()
eurusd['200 daily-EMA(Open)'] = eurusd['Open'].rolling(window=200).mean()
eurusd['50 daily-std(Open)'] = eurusd['Open'].rolling(window=50).std()
eurusd['200 daily-std(Open)'] = eurusd['Open'].rolling(window=200).std()
eurusd[['Open', '50 daily-EMA(Open)','200 daily-EMA(Open)', '50 daily-std(Open)', '200 daily-std(Open)']].plot(linewidth=2)
plt.ylim(0, 2)
plt.xlabel('Years', size=17)
plt.ylabel('Exchange Rate', size=17)
plt.autoscale(enable=True, axis='x', tight=True)#plt.axis('tight'
plt.tick_params(labelsize=14)
mpl.rcParams['figure.figsize'] = 12,6

From the chart above, it is clear that the Standard deviation is stationary, while the mean is not. The mean actually varies with the exchange rate trend

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 3
#p = d = q = range(0, 3)

# Generate all different combinations of p, q and q triplets
#pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
#seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]

#print('Examples of parameter combinations for Seasonal ARIMA...')
#print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
#print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
#print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
#print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
#warnings.filterwarnings("ignore") # specify to ignore warning messages

#for param in pdq:
    #for param_seasonal in seasonal_pdq:
      #  try:
     #       mod = sm.tsa.statespace.SARIMAX(y,
#                                            order=param,
 #                                           seasonal_order=param_seasonal,
  #                                          enforce_stationarity=False,
   #                                         enforce_invertibility=False)

    #        results = mod.fit()

 #           print('ARIMA{}x{}52 - AIC:{}'.format(param, param_seasonal, results.aic))
  #     except:
   #        continue

In [ ]:
#mod = sm.tsa.statespace.SARIMAX(eurusd.Open,
#                                order=(1, 0, 0),
 #                               seasonal_order=(2, 1, 1, 52),
  #                              enforce_stationarity=False,
   #                             enforce_invertibility=False)

#results = mod.fit()

#print(results.summary().tables[1])

In [ ]:
# Diagnostics of results
#results.plot_diagnostics(figsize=(15, 12))
#plt.show()

Our primary concern is to ensure that the residuals of our model are uncorrelated and normally distributed with zero-mean. If the seasonal ARIMA model does not satisfy these properties, it is a good indication that it can be further improved.

In this case, our model diagnostics suggests that the model residuals are normally distributed based on the following:

In the top right plot, we see that the red KDE line follows closely with the N(0,1) line (where N(0,1)) is the standard notation for a normal distribution with mean 0 and standard deviation of 1). This is a good indication that the residuals are normally distributed.
The qq-plot on the bottom left shows that the ordered distribution of residuals (blue dots) follows the linear trend of the samples taken from a standard normal distribution with N(0, 1). Again, this is a strong indication that the residuals are normally distributed.
The residuals over time (top left plot) don’t display any obvious seasonality and appear to be white noise. This is confirmed by the autocorrelation (i.e. correlogram) plot on the bottom right, which shows that the time series residuals have low correlation with lagged versions of itself.

Those observations lead us to conclude that our model produces a satisfactory fit that could help us understand our time series data and forecast future values.

Hypothesis Testing:
    
The Null hypothesis of this project states that the time series data of eurysd is non-stationary. Using the Dickey fuller test, I will determine below whether or not this is true.

In [ ]:
print('Augmented Dickey-Fuller Test on EURUSD Exchange Rate Data')
dftest = adfuller(eurusd['Open'], autolag='AIC')
dftest

In [ ]:
print('Augmented Dickey-Fuller Test on EURUSD Exchange Rate Data')
dfout = pd.Series(dftest[0:4], index = ['ADF Test Statistic', 'p-value', '# lags used', '# observations'])

for key, val in dftest[4].items():
    dfout[f'critical value({key})']=val
    print(dfout)

#### MACHINE LEARNING: SARIMAX MODEL

In [ ]:
# Use the auto arima function to generate the optimal parameter values for the Arima Model
stepwise_fit = auto_arima(eurusd['Open'], start_P = 1, start_q = 1, max_p = 2, max_q = 2, m=52, start_p = 0, seasonal =True
                          , d=None, D=1, trace=True, 
                           error_action='ignore', # We don't want to know if an order does not work
                           surpress_warnings=True, #We don't want convergence warnings
                           stepwise=True) #Set to stepwise
stepwise_fit.summary()

# AS can be observed above, the Auto Arima function generated a Model of SARIMAX(1, 0, 0)x(2, 1, 1, 52)

The P-value above is greater than 5% using the dickey fuller test, hence we accept the null-hypothesis to be true, that our time series data for eurusd is non-stationary

In [ ]:
eurusd.shape

In [ ]:
# Split Dataset into training and test using a 80% split in favor of train
size = int(len(eurusd.Open) * 0.80)
train, test = eurusd.Open[0:size], eurusd.Open[size:len(eurusd.Open)]

In [ ]:
# Import SARIMAX 

from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Fit SARIMAX model
model  = SARIMAX(train, order=(1,0,0), seasonal_order=(2,1,1,52), enforce_invertibility=False)
result = model.fit()
result.summary()

In [ ]:
# Make predictions
start = len(train)
end = len(train) + len(test)-1
predictions = results.predict(start = start, end = end, dynamic = False, typ='levels').rename('SARIMAX(1,0,0) Predictions')

In [ ]:
# Define expected
expected=test

In [ ]:
# Forcast error
for i in range(len(predictions)):
    Forecast_error = predictions[i] - test[i]

In [ ]:
# Plot predictions against known values
ax = test.plot(legend=True, figsize=(20,10), linewidth=1)
predictions.plot(legend=True, linewidth=1)
#ax.autoscale(axis = 'x', tight=True)

In [ ]:
# Compare predictions to expected values

for i in range(len(predictions)):
    print(f"predicted={predictions[i]}, \t expected={test[i]}")

In [ ]:
from sklearn.metrics import mean_squared_error

error = mean_squared_error(test, predictions)
print(f'SARIMAX(1,0,0) MSE Error {error}')

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test, predictions)
print(f'SARIMAX(1,0,0) RMSE Error: {error:11.10}')

#### Retrain Sarimax model

In [ ]:
model = SARIMAX(eurusd['Open'], order=(1,0,0), seasonal_order=(2,1,1,52), enforce_invinsibility=False)
result2 = model.fit()
fcast = result2.predict(len(eurusd), len(eurusd)+365*10, dynamic=True, typ='levels').rename('SARIMAX(1,0,0)Forecast')

In [ ]:
fcast_start = max(eurusd.index.values) + pd.Timedelta(days = 1)
fcast_end = max(eurusd.index.values) + pd.Timedelta(days = 10 * 365 + 1)
fcast = pd.DataFrame(fcast)
fcast['date'] = pd.date_range(fcast_start, fcast_end)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
ax.plot(eurusd['Open'], label = 'Open')
ax.plot(fcast.date, fcast.iloc[:, 0], label = 'forecast')
ax.legend()
ax.autoscale(axis = 'x', tight=True)
plt.show()  